# **Describe Data:**

https://www.kaggle.com/c/petfinder-adoption-prediction/data

**Problem:**

predict the speed at which a pet is adopted, based on the pet’s listing on PetFinder. Sometimes a profile represents a group of pets. In this case, the speed of adoption is determined by the speed at which all of the pets are adopted. 

**The data included text, tabular, and image data.**

## **Data Fields:**

**PetID** - Unique hash ID of pet profile

**AdoptionSpeed** - Categorical speed of adoption. Lower is faster. **This is the 
value to predict.**

**Type** - Type of animal (1 = Dog, 2 = Cat)

**Name** - Name of pet (Empty if not named)

**Age** - Age of pet when listed, in months

**Breed1** - Primary breed of pet (Refer to BreedLabels dictionary)

**Breed2** - Secondary breed of pet, if pet is of mixed breed (Refer to BreedLabels 
dictionary)

**Gender** - Gender of pet (1 = Male, 2 = Female, 3 = Mixed, if profile represents 
group of pets)

**Color1** - Color 1 of pet (Refer to ColorLabels dictionary)

**Color2** - Color 2 of pet (Refer to ColorLabels dictionary)

**Color3** - Color 3 of pet (Refer to ColorLabels dictionary)

**MaturitySize** - Size at maturity (1 = Small, 2 = Medium, 3 = Large, 4 = Extra 

Large, 0 = Not Specified)

**FurLength** - Fur length (1 = Short, 2 = Medium, 3 = Long, 0 = Not Specified)

**Vaccinated** - Pet has been vaccinated (1 = Yes, 2 = No, 3 = Not Sure)

**Dewormed** - Pet has been dewormed (1 = Yes, 2 = No, 3 = Not Sure)

**Sterilized** - Pet has been spayed / neutered (1 = Yes, 2 = No, 3 = Not Sure)

**Health** - Health Condition (1 = Healthy, 2 = Minor Injury, 3 = Serious Injury, 0 
= Not Specified)

**Quantity** - Number of pets represented in profile

**Fee** - Adoption fee (0 = Free)

**State** - State location in Malaysia (Refer to StateLabels dictionary)

**RescuerID** - Unique hash ID of rescuer

**VideoAmt** - Total uploaded videos for this pet

**PhotoAmt** - Total uploaded photos for this pet

**Description** - Profile write-up for this pet. The primary language used is English, with some in Malay or Chinese.

## **AdoptionSpeed**

The value is determined by how quickly, if at all, a pet is adopted. The values are determined in the 
following way:

**0** - Pet was adopted on the same day as it was listed.

**1** - Pet was adopted between 1 and 7 days (1st week) after being listed.

**2** - Pet was adopted between 8 and 30 days (1st month) after being listed.

**3** - Pet was adopted between 31 and 90 days (2nd & 3rd month) after being listed.

**4** - No adoption after 100 days of being listed. (There are no pets in this 
dataset that waited between 90 and 100 days).






Evaluation criteria:
- Use of tabular data = 50% of the mark 
- Use of text data (Pet Description) or Image Data = 75% of the mark 

- Use of ALL sources = Tabular+Image+Text+Sentiment = 100% of the mark 


Bonus:
- Submission to Kaggle on official Leaderboard = +5
- Top 15% score = +10
- Top 5% score = +20
- Top 3% score = +30

# **Import Needed Library**

In [ ]:
import os

In [ ]:
#libraries
import numpy as np 
import pandas as pd 
import os
import json
import seaborn as sns 
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')
import lightgbm as lgb
import xgboost as xgb
import time
import datetime
from PIL import Image
from wordcloud import WordCloud
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import mean_squared_error, roc_auc_score
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
import gc

from tqdm import tqdm_notebook
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import random
import warnings
warnings.filterwarnings("ignore")
from functools import partial
pd.set_option('max_colwidth', 500)
pd.set_option('max_columns', 500)
pd.set_option('max_rows', 100)
import os
import scipy as sp
from math import sqrt
from collections import Counter
from sklearn.metrics import confusion_matrix as sk_cmatrix

from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import TweetTokenizer
from sklearn.ensemble import RandomForestClassifier
#import langdetect
#import eli5
from IPython.display import display 

from sklearn.metrics import cohen_kappa_score
def kappa(y_true, y_pred):
  return cohen_kappa_score(y_true, y_pred, weights='quadratic')


#libraries
import numpy as np 
import pandas as pd 
import os
import json
import seaborn as sns 
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')
import lightgbm as lgb
import xgboost as xgb
import time
import datetime
from PIL import Image
from wordcloud import WordCloud
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold

from sklearn.metrics import mean_squared_error, roc_auc_score
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
import gc

from tqdm import tqdm_notebook
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import random
import warnings
warnings.filterwarnings("ignore")
from functools import partial
pd.set_option('max_colwidth', 500)
pd.set_option('max_columns', 500)
pd.set_option('max_rows', 100)
import os
import scipy as sp
from math import sqrt
from collections import Counter
from sklearn.metrics import confusion_matrix as sk_cmatrix

from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import TweetTokenizer

from sklearn.ensemble import RandomForestClassifier
#import langdetect
#import eli5
from IPython.display import display 

from sklearn.metrics import cohen_kappa_score
def kappa(y_true, y_pred):
    return cohen_kappa_score(y_true, y_pred, weights='quadratic')


# importing the required library
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# **Load and Explore Data**

In [ ]:
breeds = pd.read_csv('/content/drive/MyDrive/Datasets/p2/breed_labels.csv')
colors = pd.read_csv('/content/drive/MyDrive/Datasets/p2/color_labels.csv')
states = pd.read_csv('/content/drive/MyDrive/Datasets/p2/state_labels.csv')
train = pd.read_csv('/content/drive/MyDrive/Datasets/p2/train.csv')
test = pd.read_csv('/content/drive/MyDrive/Datasets/p2/test.csv')
sub = pd.read_csv('/content/drive/MyDrive/Datasets/p2/sample_submission.csv')

In [ ]:
train["dataset_type"]="Train"
test["dataset_type"]="Test"

In [ ]:

all_data = pd.concat([train, test])

In [ ]:
print(os.listdir())

['.config', 'drive', 'sample_data']


In [ ]:
train=pd.read_csv('/content/drive/MyDrive/Datasets/p2/train_with_image_features3.csv')

In [ ]:
train

,Unnamed: 0,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,FurLength,Vaccinated,Dewormed,Sterilized,Health,Quantity,Fee,State,RescuerID,VideoAmt,Description,PetID,PhotoAmt,AdoptionSpeed,img_feat0,img_feat1,img_feat2,img_feat3,img_feat4,img_feat5,img_feat6,img_feat7,img_feat8,img_feat9,img_feat10,img_feat11,img_feat12,img_feat13,img_feat14,img_feat15,img_feat16,img_feat17,img_feat18,img_feat19,img_feat20,img_feat21,img_feat22,img_feat23,img_feat24,img_feat25,img_feat26,img_feat27,img_feat28,img_feat29,img_feat30,img_feat31,img_feat32,img_feat33,img_feat34,img_feat35,img_feat36,img_feat37,img_feat38,img_feat39,img_feat40,img_feat41,img_feat42,img_feat43,img_feat44,img_feat45,img_feat46,img_feat47,img_feat48,img_feat49,img_feat50,img_feat51,img_feat52,img_feat53,img_feat54,img_feat55,img_feat56,img_feat57,img_feat58,img_feat59,img_feat60,img_feat61,img_feat62,img_feat63,img_feat64,img_feat65,img_feat66,img_feat67,img_feat68,img_feat69,img_feat70,img_feat71,img_feat72,img_feat73,img_feat74,img_feat75,img_feat76,img_feat77,img_feat78,img_feat79,img_feat80,img_feat81,img_feat82,img_feat83,img_feat84,img_feat85,img_feat86,img_feat87,img_feat88,img_feat89,img_feat90,img_feat91,img_feat92,img_feat93,img_feat94,img_feat95,img_feat96,img_feat97,img_feat98,img_feat99,img_feat100,img_feat101,img_feat102,img_feat103,img_feat104,img_feat105,img_feat106,img_feat107,img_feat108,img_feat109,img_feat110,img_feat111,img_feat112,img_feat113,img_feat114,img_feat115,img_feat116,img_feat117,img_feat118,img_feat119,img_feat120,img_feat121,img_feat122,img_feat123,img_feat124,img_feat125,img_feat126,img_feat127,img_feat128,img_feat129,img_feat130,img_feat131,img_feat132,img_feat133,img_feat134,img_feat135,img_feat136,img_feat137,img_feat138,img_feat139,img_feat140,img_feat141,img_feat142,img_feat143,img_feat144,img_feat145,img_feat146,img_feat147,img_feat148,img_feat149,img_feat150,img_feat151,img_feat152,img_feat153,img_feat154,img_feat155,img_feat156,img_feat157,img_feat158,img_feat159,img_feat160,img_feat161,img_feat162,img_feat163,img_feat164,img_feat165,img_feat166,img_feat167,img_feat168,img_feat169,img_feat170,img_feat171,img_feat172,img_feat173,img_feat174,img_feat175,img_feat176,img_feat177,img_feat178,img_feat179,img_feat180,img_feat181,img_feat182,img_feat183,img_feat184,img_feat185,img_feat186,img_feat187,img_feat188,img_feat189,img_feat190,img_feat191,img_feat192,img_feat193,img_feat194,img_feat195,img_feat196,img_feat197,img_feat198,img_feat199,img_feat200,img_feat201,img_feat202,img_feat203,img_feat204,img_feat205,img_feat206,img_feat207,img_feat208,img_feat209,img_feat210,img_feat211,img_feat212,img_feat213,img_feat214,img_feat215,img_feat216,img_feat217,img_feat218,img_feat219,img_feat220,img_feat221,img_feat222,img_feat223,img_feat224,img_feat225,img_feat226,img_feat227,img_feat228,img_feat229,img_feat230,img_feat231,img_feat232,img_feat233,img_feat234,img_feat235,img_feat236,img_feat237,img_feat238,img_feat239,img_feat240,img_feat241,img_feat242,img_feat243,img_feat244,img_feat245,img_feat246,img_feat247,img_feat248,img_feat249,img_feat250,img_feat251,img_feat252,img_feat253,img_feat254,img_feat255
0,0,2,Nibble,3,299,0,1,1,7,0,1,1,2,2,2,1,1,100,41326,8480853f516546f6cf33aa88cd76c379,0,Nibble is a 3+ month old ball of cuteness. He is energetic and playful. I rescued a couple of cats a few months ago but could not get them neutered in time as the clinic was fully scheduled. The result was this little kitty. I do not have enough space and funds to care for more cats in my household. Looking for responsible people to take over Nibble's care.,86e1089a3,1.0,2,0.002188,0.159618,0.038098,0.016042,0.057492,0.002475,0.006010,0.005836,0.056111,0.053242,0.091537,0.003917,0.009016,0.002808,0.018201,0.027517,0.004162,0.014599,0.003727,0.000955,0.021116,0.051682,0.473224,0.003007,0.062389,0.055852,0.570284,0.003538,0.006155,0.080830,0.017399,0.335236,0.092179,0.093450,0.227340,0.128001,0.027068,0.025152,0.869941,0.008428,0.0221

In [ ]:

test=pd.read_csv('/content/drive/MyDrive/Datasets/p2/test_with_image_features3.csv')

In [ ]:

train=train.iloc[:,1:]
test=test.iloc[:,1:]

In [ ]:

print(train.shape)
print(test.shape)

(14993, 280)
(3972, 279)


In [ ]:
train.iloc[:,24:]

,img_feat0,img_feat1,img_feat2,img_feat3,img_feat4,img_feat5,img_feat6,img_feat7,img_feat8,img_feat9,img_feat10,img_feat11,img_feat12,img_feat13,img_feat14,img_feat15,img_feat16,img_feat17,img_feat18,img_feat19,img_feat20,img_feat21,img_feat22,img_feat23,img_feat24,img_feat25,img_feat26,img_feat27,img_feat28,img_feat29,img_feat30,img_feat31,img_feat32,img_feat33,img_feat34,img_feat35,img_feat36,img_feat37,img_feat38,img_feat39,img_feat40,img_feat41,img_feat42,img_feat43,img_feat44,img_feat45,img_feat46,img_feat47,img_feat48,img_feat49,img_feat50,img_feat51,img_feat52,img_feat53,img_feat54,img_feat55,img_feat56,img_feat57,img_feat58,img_feat59,img_feat60,img_feat61,img_feat62,img_feat63,img_feat64,img_feat65,img_feat66,img_feat67,img_feat68,img_feat69,img_feat70,img_feat71,img_feat72,img_feat73,img_feat74,img_feat75,img_feat76,img_feat77,img_feat78,img_feat79,img_feat80,img_feat81,img_feat82,img_feat83,img_feat84,img_feat85,img_feat86,img_feat87,img_feat88,img_feat89,img_feat90,img_feat91,img_feat92,img_feat93,img_feat94,img_feat95,img_feat96,img_feat97,img_feat98,img_feat99,img_feat100,img_feat101,img_feat102,img_feat103,img_feat104,img_feat105,img_feat106,img_feat107,img_feat108,img_feat109,img_feat110,img_feat111,img_feat112,img_feat113,img_feat114,img_feat115,img_feat116,img_feat117,img_feat118,img_feat119,img_feat120,img_feat121,img_feat122,img_feat123,img_feat124,img_feat125,img_feat126,img_feat127,img_feat128,img_feat129,img_feat130,img_feat131,img_feat132,img_feat133,img_feat134,img_feat135,img_feat136,img_feat137,img_feat138,img_feat139,img_feat140,img_feat141,img_feat142,img_feat143,img_feat144,img_feat145,img_feat146,img_feat147,img_feat148,img_feat149,img_feat150,img_feat151,img_feat152,img_feat153,img_feat154,img_feat155,img_feat156,img_feat157,img_feat158,img_feat159,img_feat160,img_feat161,img_feat162,img_feat163,img_feat164,img_feat165,img_feat166,img_feat167,img_feat168,img_feat169,img_feat170,img_feat171,img_feat172,img_feat173,img_feat174,img_feat175,img_feat176,img_feat177,img_feat178,img_feat179,img_feat180,img_feat181,img_feat182,img_feat183,img_feat184,img_feat185,img_feat186,img_feat187,img_feat188,img_feat189,img_feat190,img_feat191,img_feat192,img_feat193,img_feat194,img_feat195,img_feat196,img_feat197,img_feat198,img_feat199,img_feat200,img_feat201,img_feat202,img_feat203,img_feat204,img_feat205,img_feat206,img_feat207,img_feat208,img_feat209,img_feat210,img_feat211,img_feat212,img_feat213,img_feat214,img_feat215,img_feat216,img_feat217,img_feat218,img_feat219,img_feat220,img_feat221,img_feat222,img_feat223,img_feat224,img_feat225,img_feat226,img_feat227,img_feat228,img_feat229,img_feat230,img_feat231,img_feat232,img_feat233,img_feat234,img_feat235,img_feat236,img_feat237,img_feat238,img_feat239,img_feat240,img_feat241,img_feat242,img_feat243,img_feat244,img_feat245,img_feat246,img_feat247,img_feat248,img_feat249,img_feat250,img_feat251,img_feat252,img_feat253,img_feat254,img_feat255
0,0.002188,0.159618,0.038098,0.016042,0.057492,0.002475,0.006010,0.005836,0.056111,0.053242,0.091537,0.003917,0.009016,0.002808,0.018201,0.027517,0.004162,0.014599,0.003727,0.000955,0.021116,0.051682,0.473224,0.003007,0.062389,0.055852,0.570284,0.003538,0.006155,0.080830,0.017399,0.335236,0.092179,0.093450,0.227340,0.128001,0.027068,0.025152,0.869941,0.008428,0.022113,0.058101,0.143517,0.001105,0.016569,0.006358,0.002855,0.017127,0.007376,0.105682,0.018531,0.058405,0.140650,0.095219,0.000487,0.002187,0.001383,0.032473,0.375653,0.005795,0.035427,0.008591,0.000401,0.008760,0.080938,0.311239,0.001677,0.003744,0.000752,0.036551,0.015445,0.057837,0.001342,0.012648,0.002054,0.142428,0.220539,0.169896,0.004395,0.030933,0.003468,0.003728,0.041504,0.027412,0.004538,0.001779,0.056929,0.034483,0.152255,0.041409,0.010441,0.008956,0.027596,0.001817,0.544008,0.006841,0.066498,0.005976,0.001038,0.011277,0.009696,0.001707,0.133683,0.010067,0.002030,0.006807,0.457843,0.001219,0.026212,0.545816,0.009249,0.084993,0.395695,0.016416,0.022992,

In [ ]:

images_features = train.iloc[:,24:].columns.to_list()

In [ ]:


train['Name'] = train['Name'].fillna('No_name_found')
test['Name'] = test['Name'].fillna('No_name_found')
#all_data['Name'] = all_data['Name'].fillna('No_name_found')

train['No_name'] = 0
train.loc[train['Name'] == 'No_name_found', 'No_name'] = 1
test['No_name'] = 0
test.loc[test['Name'] == 'No_name_found', 'No_name'] = 1
#all_data['No_name'] = 0
#all_data.loc[all_data['Name'] == 'No_name_found', 'No_name'] = 1


train['Pure_breed'] = 0
train.loc[train['Breed2'] == 0, 'Pure_breed'] = 1

test['Pure_breed'] = 0
test.loc[test['Breed2'] == 0, 'Pure_breed'] = 1

#all_data['Pure_breed'] = 0
#all_data.loc[all_data['Breed2'] == 0, 'Pure_breed'] = 1

In [ ]:
train.shape

(14993, 282)

In [ ]:
print(train.shape)
print(test.shape)

(14993, 282)
(3972, 281)


In [ ]:
#drop unnedded columns
train.isnull().sum()

Type           0
Name           0
Age            0
Breed1         0
Breed2         0
              ..
img_feat253    0
img_feat254    0
img_feat255    0
No_name        0
Pure_breed     0
Length: 282, dtype: int64

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder

from keras.utils import to_categorical

In [ ]:


#label encoding
Labelencoder = LabelEncoder()
encoder = OrdinalEncoder()

In [ ]:


train=train.drop(['Name','RescuerID', 'Description','State','PetID' ], axis = 1)
train=train.drop(['Breed1','Breed2'], axis = 1)

test=test.drop(['Name','RescuerID', 'Description','State',"PetID" ], axis = 1)
test=test.drop(['Breed1','Breed2'], axis = 1)

In [ ]:
print(train.shape)
print(test.shape)

(14993, 275)
(3972, 274)


In [ ]:
# Encoding data
train = pd.get_dummies(train, columns=["Gender"], prefix=["Gender_is"])
train = pd.get_dummies(train, columns=["Color1"], prefix=["Color1_is"])
train = pd.get_dummies(train, columns=["Color2"], prefix=["Color2_is"])
train = pd.get_dummies(train, columns=["Color3"], prefix=["Color3_is"])

# Encoding data
test = pd.get_dummies(test, columns=["Gender"], prefix=["Gender_is"])
test = pd.get_dummies(test, columns=["Color1"], prefix=["Color1_is"])
test = pd.get_dummies(test, columns=["Color2"], prefix=["Color2_is"])
test = pd.get_dummies(test, columns=["Color3"], prefix=["Color3_is"])

print(train.shape)
print(test.shape)

(14993, 294)
(3972, 293)


In [ ]:
train = pd.get_dummies(train, columns=["MaturitySize"], prefix=["MaturitySize_is"])
train = pd.get_dummies(train, columns=["FurLength"], prefix=["FurLength_is"])
train = pd.get_dummies(train, columns=["Vaccinated"], prefix=["Vaccinated_is"])


test = pd.get_dummies(test, columns=["MaturitySize"], prefix=["MaturitySize_is"])
test = pd.get_dummies(test, columns=["FurLength"], prefix=["FurLength_is"])
test = pd.get_dummies(test, columns=["Vaccinated"], prefix=["Vaccinated_is"])

print(train.shape)
print(test.shape)

(14993, 301)
(3972, 300)


In [ ]:
train = pd.get_dummies(train, columns=["Dewormed"], prefix=["Dewormed_is"])
test = pd.get_dummies(test, columns=["Dewormed"], prefix=["Dewormed_is"])

print(train.shape)
print(test.shape)

(14993, 303)
(3972, 302)


In [ ]:
train = pd.get_dummies(train, columns=["Sterilized"], prefix=["Sterilized_is"])
test = pd.get_dummies(test, columns=["Sterilized"], prefix=["Sterilized_is"])

print(train.shape)
print(test.shape)

(14993, 305)
(3972, 304)


In [ ]:
#train = pd.get_dummies(train, columns=["State"], prefix=["State_is"])
#test = pd.get_dummies(test, columns=["State"], prefix=["State_is"])

print(train.shape)
print(test.shape)

(14993, 305)
(3972, 304)


In [ ]:
train["Health"] = encoder.fit_transform(train["Health"].values.reshape(-1, 1))
test["Health"] = encoder.fit_transform(test["Health"].values.reshape(-1, 1))

print(train.shape)
print(test.shape)

(14993, 305)
(3972, 304)


In [ ]:
t1=train.columns
t2=test.columns

In [ ]:
import numpy as np

main_list = np.setdiff1d(t1,t2)

In [ ]:
main_list

array(['AdoptionSpeed'], dtype=object)

In [ ]:
train.iloc[:, 7]

0        2
1        0
2        3
3        2
4        2
        ..
14988    2
14989    4
14990    3
14991    4
14992    3
Name: AdoptionSpeed, Length: 14993, dtype: int64

In [ ]:

train.iloc[:, 7]
v=train.iloc[:, 7] #get the values of the last column
train=train.drop(['AdoptionSpeed'],axis=1)  #drop the column

In [ ]:

train.shape

(14993, 304)

In [ ]:

train.insert(304,'AdoptionSpeed',v) #insert as the first column

In [ ]:
train.head(1)

,Type,Age,Health,Quantity,Fee,VideoAmt,PhotoAmt,img_feat0,img_feat1,img_feat2,img_feat3,img_feat4,img_feat5,img_feat6,img_feat7,img_feat8,img_feat9,img_feat10,img_feat11,img_feat12,img_feat13,img_feat14,img_feat15,img_feat16,img_feat17,img_feat18,img_feat19,img_feat20,img_feat21,img_feat22,img_feat23,img_feat24,img_feat25,img_feat26,img_feat27,img_feat28,img_feat29,img_feat30,img_feat31,img_feat32,img_feat33,img_feat34,img_feat35,img_feat36,img_feat37,img_feat38,img_feat39,img_feat40,img_feat41,img_feat42,img_feat43,img_feat44,img_feat45,img_feat46,img_feat47,img_feat48,img_feat49,img_feat50,img_feat51,img_feat52,img_feat53,img_feat54,img_feat55,img_feat56,img_feat57,img_feat58,img_feat59,img_feat60,img_feat61,img_feat62,img_feat63,img_feat64,img_feat65,img_feat66,img_feat67,img_feat68,img_feat69,img_feat70,img_feat71,img_feat72,img_feat73,img_feat74,img_feat75,img_feat76,img_feat77,img_feat78,img_feat79,img_feat80,img_feat81,img_feat82,img_feat83,img_feat84,img_feat85,img_feat86,img_feat87,img_feat88,img_feat89,img_feat90,img_feat91,img_feat92,img_feat93,img_feat94,img_feat95,img_feat96,img_feat97,img_feat98,img_feat99,img_feat100,img_feat101,img_feat102,img_feat103,img_feat104,img_feat105,img_feat106,img_feat107,img_feat108,img_feat109,img_feat110,img_feat111,img_feat112,img_feat113,img_feat114,img_feat115,img_feat116,img_feat117,img_feat118,img_feat119,img_feat120,img_feat121,img_feat122,img_feat123,img_feat124,img_feat125,img_feat126,img_feat127,img_feat128,img_feat129,img_feat130,img_feat131,img_feat132,img_feat133,img_feat134,img_feat135,img_feat136,img_feat137,img_feat138,img_feat139,img_feat140,img_feat141,img_feat142,img_feat143,img_feat144,img_feat145,img_feat146,img_feat147,img_feat148,img_feat149,img_feat150,img_feat151,img_feat152,img_feat153,img_feat154,img_feat155,img_feat156,img_feat157,img_feat158,img_feat159,img_feat160,img_feat161,img_feat162,img_feat163,img_feat164,img_feat165,img_feat166,img_feat167,img_feat168,img_feat169,img_feat170,img_feat171,img_feat172,img_feat173,img_feat174,img_feat175,img_feat176,img_feat177,img_feat178,img_feat179,img_feat180,img_feat181,img_feat182,img_feat183,img_feat184,img_feat185,img_feat186,img_feat187,img_feat188,img_feat189,img_feat190,img_feat191,img_feat192,img_feat193,img_feat194,img_feat195,img_feat196,img_feat197,img_feat198,img_feat199,img_feat200,img_feat201,img_feat202,img_feat203,img_feat204,img_feat205,img_feat206,img_feat207,img_feat208,img_feat209,img_feat210,img_feat211,img_feat212,img_feat213,img_feat214,img_feat215,img_feat216,img_feat217,img_feat218,img_feat219,img_feat220,img_feat221,img_feat222,img_feat223,img_feat224,img_feat225,img_feat226,img_feat227,img_feat228,img_feat229,img_feat230,img_feat231,img_feat232,img_feat233,img_feat234,img_feat235,img_feat236,img_feat237,img_feat238,img_feat239,img_feat240,img_feat241,img_feat242,img_feat243,img_feat244,img_feat245,img_feat246,img_feat247,img_feat248,img_feat249,img_feat250,img_feat251,img_feat252,img_feat253,img_feat254,img_feat255,No_name,Pure_breed,Gender_is_1,Gender_is_2,Gender_is_3,Color1_is_1,Color1_is_2,Color1_is_3,Color1_is_4,Color1_is_5,Color1_is_6,Color1_is_7,Color2_is_0,Color2_is_2,Color2_is_3,Color2_is_4,Color2_is_5,Color2_is_6,Color2_is_7,Color3_is_0,Color3_is_3,Color3_is_4,Color3_is_5,Color3_is_6,Color3_is_7,MaturitySize_is_1,MaturitySize_is_2,MaturitySize_is_3,MaturitySize_is_4,FurLength_is_1,FurLength_is_2,FurLength_is_3,Vaccinated_is_1,Vaccinated_is_2,Vaccinated_is_3,Dewormed_is_1,Dewormed_is_2,Dewormed_is_3,Sterilized_is_1,Sterilized_is_2,Sterilized_is_3,AdoptionSpeed
0,2,3,0.0,1,100,0,1.0,0.002188,0.159618,0.038098,0.016042,0.057492,0.002475,0.00601,0.005836,0.056111,0.053242,0.091537,0.003917,0.009016,0.002808,0.018201,0.027517,0.004162,0.014599,0.003727,0.000955,0.021116,0.051682,0.473224,0.003007,0.062389,0.055852,0.570284,0.003538,0.006155,0.08083,0.017399,0.335236,0.092179,0.09345,0.22734,0.128001,0.027068,0.025152,0.869941,0.008428,0.022113,0.058101,0.143517,0.001105,0.016569,0.0

In [ ]:
train.shape

(14993, 305)

In [ ]:
test.shape

(3972, 304)

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
label = "AdoptionSpeed"

In [ ]:

column_names_list = train.iloc[:,:-1].columns

In [ ]:

new_cols = column_names_list.tolist()

In [ ]:
#new_cols.remove('PetID')

In [ ]:

for i in images_features:
  new_cols.remove(i)

In [ ]:


train_data=train.copy()

In [ ]:

numeric_f = new_cols
train_data[numeric_f] = train_data[numeric_f].apply(lambda x: (x-x.mean())/ x.std(), axis=0)

In [ ]:
train_data

,Type,Age,Health,Quantity,Fee,VideoAmt,PhotoAmt,img_feat0,img_feat1,img_feat2,img_feat3,img_feat4,img_feat5,img_feat6,img_feat7,img_feat8,img_feat9,img_feat10,img_feat11,img_feat12,img_feat13,img_feat14,img_feat15,img_feat16,img_feat17,img_feat18,img_feat19,img_feat20,img_feat21,img_feat22,img_feat23,img_feat24,img_feat25,img_feat26,img_feat27,img_feat28,img_feat29,img_feat30,img_feat31,img_feat32,img_feat33,img_feat34,img_feat35,img_feat36,img_feat37,img_feat38,img_feat39,img_feat40,img_feat41,img_feat42,img_feat43,img_feat44,img_feat45,img_feat46,img_feat47,img_feat48,img_feat49,img_feat50,img_feat51,img_feat52,img_feat53,img_feat54,img_feat55,img_feat56,img_feat57,img_feat58,img_feat59,img_feat60,img_feat61,img_feat62,img_feat63,img_feat64,img_feat65,img_feat66,img_feat67,img_feat68,img_feat69,img_feat70,img_feat71,img_feat72,img_feat73,img_feat74,img_feat75,img_feat76,img_feat77,img_feat78,img_feat79,img_feat80,img_feat81,img_feat82,img_feat83,img_feat84,img_feat85,img_feat86,img_feat87,img_feat88,img_feat89,img_feat90,img_feat91,img_feat92,img_feat93,img_feat94,img_feat95,img_feat96,img_feat97,img_feat98,img_feat99,img_feat100,img_feat101,img_feat102,img_feat103,img_feat104,img_feat105,img_feat106,img_feat107,img_feat108,img_feat109,img_feat110,img_feat111,img_feat112,img_feat113,img_feat114,img_feat115,img_feat116,img_feat117,img_feat118,img_feat119,img_feat120,img_feat121,img_feat122,img_feat123,img_feat124,img_feat125,img_feat126,img_feat127,img_feat128,img_feat129,img_feat130,img_feat131,img_feat132,img_feat133,img_feat134,img_feat135,img_feat136,img_feat137,img_feat138,img_feat139,img_feat140,img_feat141,img_feat142,img_feat143,img_feat144,img_feat145,img_feat146,img_feat147,img_feat148,img_feat149,img_feat150,img_feat151,img_feat152,img_feat153,img_feat154,img_feat155,img_feat156,img_feat157,img_feat158,img_feat159,img_feat160,img_feat161,img_feat162,img_feat163,img_feat164,img_feat165,img_feat166,img_feat167,img_feat168,img_feat169,img_feat170,img_feat171,img_feat172,img_feat173,img_feat174,img_feat175,img_feat176,img_feat177,img_feat178,img_feat179,img_feat180,img_feat181,img_feat182,img_feat183,img_feat184,img_feat185,img_feat186,img_feat187,img_feat188,img_feat189,img_feat190,img_feat191,img_feat192,img_feat193,img_feat194,img_feat195,img_feat196,img_feat197,img_feat198,img_feat199,img_feat200,img_feat201,img_feat202,img_feat203,img_feat204,img_feat205,img_feat206,img_feat207,img_feat208,img_feat209,img_feat210,img_feat211,img_feat212,img_feat213,img_feat214,img_feat215,img_feat216,img_feat217,img_feat218,img_feat219,img_feat220,img_feat221,img_feat222,img_feat223,img_feat224,img_feat225,img_feat226,img_feat227,img_feat228,img_feat229,img_feat230,img_feat231,img_feat232,img_feat233,img_feat234,img_feat235,img_feat236,img_feat237,img_feat238,img_feat239,img_feat240,img_feat241,img_feat242,img_feat243,img_feat244,img_feat245,img_feat246,img_feat247,img_feat248,img_feat249,img_feat250,img_feat251,img_feat252,img_feat253,img_feat254,img_feat255,No_name,Pure_breed,Gender_is_1,Gender_is_2,Gender_is_3,Color1_is_1,Color1_is_2,Color1_is_3,Color1_is_4,Color1_is_5,Color1_is_6,Color1_is_7,Color2_is_0,Color2_is_2,Color2_is_3,Color2_is_4,Color2_is_5,Color2_is_6,Color2_is_7,Color3_is_0,Color3_is_3,Color3_is_4,Color3_is_5,Color3_is_6,Color3_is_7,MaturitySize_is_1,MaturitySize_is_2,MaturitySize_is_3,MaturitySize_is_4,FurLength_is_1,FurLength_is_2,FurLength_is_3,Vaccinated_is_1,Vaccinated_is_2,Vaccinated_is_3,Dewormed_is_1,Dewormed_is_2,Dewormed_is_3,Sterilized_is_1,Sterilized_is_2,Sterilized_is_3,AdoptionSpeed
0,1.088656,-0.410452,-0.183512,-0.391224,1.004151,-0.163958,-0.828375,0.002188,0.159618,0.038098,0.016042,0.057492,0.002475,0.006010,0.005836,0.056111,0.053242,0.091537,0.003917,0.009016,0.002808,0.018201,0.027517,0.004162,0.014599,0.003727,0.000955,0.021116,0.051682,0.473224,0.003007,0.062389,0.055852,0.570284,0.003538,0.006155,0.080830,0.017399,0.335236,0.092179,0.093450,0.227340,0.128001,0.027068,0.025152,0.869941,0.008

In [ ]:
#train_data=train.copy()

In [ ]:
kfold = StratifiedKFold(n_splits=5, random_state=22)

In [ ]:
models = []
predictions = []
ids = []
result= []

In [ ]:
#features = [c for c in train_data.columns if c not in set(non_features+[label])]; features

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
from plotly.offline import iplot,init_notebook_mode 
from plotly.tools import make_subplots
from lightgbm.sklearn import LGBMRegressor
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import StratifiedKFold, train_test_split

In [ ]:

X=train_data.iloc[:,:-1]
Y=train_data.iloc[:,-1]

In [ ]:


X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.3,random_state=0)

# size of train data set
print(X_train.shape)
# size of train data set
print(Y_train.shape)

# size of test data set
print(X_test.shape)
# size of train data set
print(Y_test.shape)

(10495, 304)
(10495,)
(4498, 304)
(4498,)


In [ ]:


models = []
predictions = []
ids = []
result= []

# Train LightGBM model

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import cohen_kappa_score
import lightgbm as lgb
import time

param = {'num_leaves': 38,
         'min_data_in_leaf': 146, 
         'objective':'multiclass',
         'num_class': 5,
         'max_depth': 4,
         'learning_rate': 0.01,
         "boosting": "gbdt",
         "feature_fraction": 0.9980062052116254,
         "bagging_freq": 1,
         "bagging_fraction": 0.844212672233457,
         "bagging_seed": 11,
         "metric": 'multi_logloss',
         "lambda_l1": 0.12757257166471625,
         "random_state": 133,
         "verbosity": -1}


In [ ]:

folds = KFold(n_splits=5, shuffle=True, random_state=15)
oof = np.zeros(len(X_train))
start = time.time()
feature_importance_df = pd.DataFrame()

In [ ]:

for fold_, (trn_idx, val_idx) in enumerate(folds.split(X_train.values, Y_train.values)):
    print("fold n°{}".format(fold_))
    trn_data = lgb.Dataset(X_train.iloc[trn_idx],
                           label=Y_train.iloc[trn_idx]
                          )
    val_data = lgb.Dataset(X_train.iloc[val_idx],
                           label=Y_train.iloc[val_idx]
                          )

    num_round = 10000
    clf = lgb.train(param,
                        trn_data,
                        10000,
                        valid_sets = [trn_data, val_data],
                        verbose_eval=500,
                        early_stopping_rounds = 200)
        
    oof[val_idx] = (pd.DataFrame(clf.predict(X_train.iloc[val_idx], 
                                                     num_iteration=clf.best_iteration)).idxmax(axis=1))

print("CV score: {:<8.5f}".format(cohen_kappa_score(oof, Y_train, weights="quadratic")))

fold n°0
Training until validation scores don't improve for 200 rounds.
[500]	training's multi_logloss: 1.19363	valid_1's multi_logloss: 1.35966
[1000]	training's multi_logloss: 1.06256	valid_1's multi_logloss: 1.35086
[1500]	training's multi_logloss: 0.963165	valid_1's multi_logloss: 1.35036
Early stopping, best iteration is:
[1335]	training's multi_logloss: 0.993615	valid_1's multi_logloss: 1.34934
fold n°1
Training until validation scores don't improve for 200 rounds.
[500]	training's multi_logloss: 1.18813	valid_1's multi_logloss: 1.38342
[1000]	training's multi_logloss: 1.06102	valid_1's multi_logloss: 1.37535
Early stopping, best iteration is:
[1172]	training's multi_logloss: 1.02527	valid_1's multi_logloss: 1.37466
fold n°2
Training until validation scores don't improve for 200 rounds.
[500]	training's multi_logloss: 1.19566	valid_1's multi_logloss: 1.36271
[1000]	training's multi_logloss: 1.06729	valid_1's multi_logloss: 1.35407
[1500]	training's multi_logloss: 0.967372	valid_1

In [ ]:

clf.predict(X_test, axis=1)

array([[0.017523  , 0.34154328, 0.4254729 , 0.12496668, 0.09049414],
       [0.03356925, 0.14642356, 0.32703178, 0.25625173, 0.23672369],
       [0.00801327, 0.06801793, 0.10873784, 0.17591356, 0.6393174 ],
       ...,
       [0.02885048, 0.28179898, 0.27955411, 0.27281119, 0.13698524],
       [0.01558811, 0.15166519, 0.23610876, 0.22559218, 0.37104575],
       [0.03048966, 0.211442  , 0.4812539 , 0.1922034 , 0.08461104]])

In [ ]:
p = pd.DataFrame(clf.predict(X_test, axis=1),num_iteration=clf.best_iteration)

In [ ]:
clf.best_iteration

In [ ]:
p = (pd.DataFrame(clf.predict(X_test,num_iteration=clf.best_iteration)).idxmax(axis=1))

In [ ]:
test['AdoptionSpeed'] = p
test[['PetID', 'AdoptionSpeed']].to_csv("submission_lgbm_bayes_optimization.csv", index=False)